In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1621908624911_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

In [2]:
import re
from pyspark.sql.functions import col

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
CORPORA_PATH = 's3://cpsc5330s21/data-input/textcorpora-SMALL/'
RE_WORDS = re.compile(r'[a-z]+')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def extract_tokens(rec):
    doc_id = rec[0].split('/')[-1].split('.')[0]
    return [(doc_id, tok) for tok in RE_WORDS.findall(rec[1].lower())]


def indexDocuments(path):

    doc_tokens = sc \
        .wholeTextFiles(path) \
        .flatMap(extract_tokens) \
        .toDF(schema=['docid', 'token'])

    doc_token_count = doc_tokens \
        .groupBy("docid", "token") \
        .count()

    doc_size = doc_token_count \
        .groupBy("docid") \
        .sum("count") \
        .withColumnRenamed("sum(count)", "doc_size")

    token_docs = doc_token_count \
        .groupBy("token") \
        .count() \
        .withColumnRenamed("count", "token_ndocs")

    tfidf = doc_token_count \
        .join(doc_size, on="docid") \
        .join(token_docs, on="token") \
        .withColumn("tfidf", 1000000 * (col("count") / col("doc_size")) / col("token_ndocs")) \
        .select(col("token"), col("docid"), col("tfidf")) \
        .sort(col("tfidf").desc())
    
    return tfidf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
myindex = indexDocuments(CORPORA_PATH)
myindex.persist()
myindex.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------+-----------------+
|token|        docid|            tfidf|
+-----+-------------+-----------------+
|alice|carroll-alice|14560.08779952442|
|  the|    bible-kjv|8085.325102735142|
+-----+-------------+-----------------+
only showing top 2 rows

In [10]:
def relevance(query, df, top_n=5):
    tokens = [[tok] for tok in RE_WORDS.findall(query.lower())]
    if not tokens:
        return []
    df_query = sc.parallelize(tokens).toDF(schema=['token'])
    return df \
        .join(df_query, on="token") \
        .groupBy("docid").sum("tfidf") \
        .withColumn("rank", col("sum(tfidf)") / len(tokens)) \
        .select(col("docid"), col("rank")) \
        .sort(col("rank").desc()) \
        .take(top_n)


def pprint_df(data):
    for (i, row) in enumerate(data, 1):
        print("%d. %-20s %.4f" % (i, row.docid, row.rank))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
for query in ["buster, whale, king, and alice the rabbit!",
               "Take a whale to lunch this week!",
              "What would Jesus do about that?",
              "My name is Buster.  Deal with it.",
              "Bodice ripper?",
              "What are leaves of grass anyway?",
              "??!?"]:
    print("\nQuery:", query)
    pprint_df(relevance(query, myindex))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Query: buster, whale, king, and alice the rabbit!
1. carroll-alice        3519.2976
2. bible-kjv            2138.8566
3. burgess-busterbrown  2059.6192
4. whitman-leaves       1750.3618
5. melville-moby_dick   1636.6326

Query: Take a whale to lunch this week!
1. melville-moby_dick   980.3922
2. austen-emma          799.5355
3. carroll-alice        795.4219
4. chesterton-brown     761.3356
5. burgess-busterbrown  693.6255

Query: What would Jesus do about that?
1. burgess-busterbrown  513.6672
2. shakespeare-caesar   483.0802
3. carroll-alice        435.3393
4. austen-emma          418.2744
5. bible-kjv            395.8176

Query: My name is Buster.  Deal with it.
1. burgess-busterbrown  1488.0874
2. shakespeare-caesar   568.4194
3. austen-emma          521.7428
4. carroll-alice        506.5478
5. blake-poems          485.3013

Query: Bodice ripper?

Query: What are leaves of grass anyway?
1. bible-kjv            814.8655
2. whitman-leaves       726.1249
3. melville-moby_dick   600.69